# Importing functions and loading the data set

In [1]:
import numpy as np
import os
import cv2
import sys
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [2]:
def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, file), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    
    return images

path = "data/sample_dataset/"
images = load_images_from_folder(path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/sample_dataset/'

In [ ]:
plt.imshow(images[0], cmap="gray")
plt.show()

In [4]:
def load_data():
    with np.load("data/train_data_label.npz") as data:
        train_data = data["train_data"]
        train_label = data["train_label"]
    
    with np.load("data/test_data_label.npz") as data:
        test_data = data["test_data"]
        test_label = data["test_label"]

    return train_data, train_label, test_data, test_label

X_train, y_train, X_test, y_test = load_data()

In [5]:
SVC = make_pipeline(PCA(n_components = 0.91), SVC(kernel = "rbf"))
SVC.fit(X_train, y_train)

print(SVC.score(X_train, y_train))

y_pred = SVC.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.9998178838098707
0.8653095370886782


# Writing a function to find the top-left corner of each hand based on edge detection

In [6]:
def hand_finder(images):
    hands = []
    
    if len(images) != 28:
        for image in images:    
            img_result = []
            
            edged_image = cv2.Canny(image, 100, 200)

            i = 0

            while i < 191:
                if edged_image[0][i] + edged_image[0][i+1] + edged_image[0][i+2] + edged_image[0][i+3] + \
                edged_image[0][i+4] + edged_image[0][i+5] + edged_image[0][i+6] + edged_image[0][i+7] + \
                edged_image[0][i+8] + edged_image[0][i+9] == 0:
                    img_result.append(i)
                    i += 27
                i += 1
            
            hands.append(img_result)

    else:
        edged_image = cv2.Canny(images, 100,200)

        i = 0

        while i < 191:
            if edged_image[0][i] + edged_image[0][i+1] + edged_image[0][i+2] + edged_image[0][i+3] + \
            edged_image[0][i+4] + edged_image[0][i+5] + edged_image[0][i+6] + edged_image[0][i+7] + \
            edged_image[0][i+8] + edged_image[0][i+9] == 0:
                hands.append(i)
                i += 27
            i += 1

    return hands
    
hands_found = hand_finder(images)

<ipython-input-6-59d74303709e>:13: RuntimeWarning: overflow encountered in ubyte_scalars
  if edged_image[0][i] + edged_image[0][i+1] + edged_image[0][i+2] + edged_image[0][i+3] + \


# Writing a function to extract all values corresponding to each hand in every image

In [7]:
def hand_locator(images, hands):
    all_imgs = []

    for idx, val in enumerate(hands):
        imgs = []

        for j in val:
            img = []
            
            if j <= 3:
                if j <= 171:
                    for i in range(5):
                        hand = images[idx][:28, j+i:j+i+28]
                        img.append(hand)
                else:
                    for i in range(5):
                        hand = np.ones(28*28).reshape(28,28)
                        img.append(hand)
            else:
                if j <= 171:
                    for i in range(-3,2):
                        hand = images[idx][:28, j+i:j+i+28]
                        img.append(hand)
                else: 
                    for i in range(5):
                        hand = np.ones(28*28).reshape(28,28)
                        img.append(hand)

            imgs.append(img)

        all_imgs.append(imgs)

    return all_imgs

all_imgs = hand_locator(images, hands_found)


# Writing a function that creates 5 predictions for each hand

In [8]:
def predict(images):
    predictions = []
    
    for image in images:
        img = []
        
        for hand in image:
            hands = []
            
            for diff in hand:
                if diff.shape[1] != 28:
                    break
                else:
                    pred = SVC.predict(diff.reshape(1,784))
                    hands.append(str(pred[0]))
            
            img.append(hands)
            
        predictions.append(img)
        
    return predictions

predictions = predict(all_imgs)


In [9]:
def beautifier(predictions):
    final = [] 
    for i in predictions:
        k = []
        
        for j in range(5):
            y = []
            
            for x in range(len(i)):
                y.append(i[x][j])
            
            for f in range(len(y)):
                y[f] = y[f].zfill(2)
            y = "".join(y)
            
            k.append(y)
            
        final.append(k)
   
    return final

beautified = beautifier(predictions)
    

# Saving the actual labels of each image to a list

In [10]:
def load_true_labels(folder):
    labels = []
    
    for file in os.listdir(folder):
        labels.append(file.split(".")[0])
        
    return labels

path = "data/sample_dataset/"
true_labels = load_true_labels(path)

In [11]:
print(true_labels)

['0418182407', '152202', '151124', '180019', '1305181420', '08200705', '24131115', '2406', '170204', '0008240417', '08232304', '10081706', '021514', '231807', '0219', '070018', '08000513', '18010222', '19221623', '24220520', '19181012', '2423142413', '0114032020', '06160712', '21230615']


# Checking the predictions agains the true labels and calculating accuracy

In [12]:
correct = 0
incorrect = 0

for i in range(len(beautified)):
    for j in range(len(beautified[i])):
        if beautified[i][j] == true_labels[i]:
            correct += 1
            break
        else:
            incorrect += 1

In [13]:
print(f"Number correct: {correct} \nAccuracy: {correct/len(true_labels)}")


Number correct: 15 
Accuracy: 0.6


# Making predictions for al 10,000 images in the full data set

In [14]:
import time
start = time.time()

In [15]:
dataset = np.load("data/test_images_task2.npy")
dataset = dataset.astype('uint8')
print(dataset.shape)

(10000, 28, 200)


In [16]:
hands_found_dataset = hand_finder(dataset)
all_img_dataset = hand_locator(dataset, hands_found_dataset)
dataset_preds = predict(all_img_dataset)

<ipython-input-6-59d74303709e>:13: RuntimeWarning: overflow encountered in ubyte_scalars
  if edged_image[0][i] + edged_image[0][i+1] + edged_image[0][i+2] + edged_image[0][i+3] + \


In [17]:
end = time.time()
print(f"It took {round(end-start)} seconds to make predictions for all 10,000 images. \nThis is {round((end-start)/60)} minute(s).")


It took 325 seconds to make predictions for all 10,000 images. 
This is 5 minute(s).


In [18]:
dataset_final_predictions = beautifier(dataset_preds)